In [12]:
# mporting the libraries
import torch
import pandas as pd
import numpy as np

from torch.utils.data import DataLoader
from tqdm import tqdm
import albumentations  as A
from albumentations.pytorch.transforms import ToTensorV2
from torch.utils.data import Dataset

import sys
sys.path.insert(0, '../dataloader/')
from datasetCOVIDx import ImageDataset, LungImageDataset


# compute Mean and std of COVIDx CXR3 dataset

In [13]:
train_txt= pd.read_csv('/mnt/DATA/research/classificationCOVID19applyseg/dataset/COVIDxCXR3/train.txt', sep= '\s+', header=None)
train_txt.columns= ["patient id","file_name","class","source"]
train_txt['label'] = np.where(train_txt['class']== 'negative', 0, 1) # tao cot label theo dk cua class
# chi lay cot filename va label (1,4)
train_txt1 = train_txt.iloc[:,[1,4]]
train_txt1

,file_name,label
0,ARDSSevere.png,0
1,acute-respiratory-distress-syndrome-ards-1.jpg,0
2,acute-respiratory-distress-syndrome-ards.jpg,0
3,ards-secondary-to-tiger-snake-bite.png,0
4,pneumocystis-pneumonia-2-PA.png,0
...,...,...
29981,55204bb8-89fc-449d-8563-e91654abea9d.png,0
29982,0f4b80b6-fd39-451f-aebf-7cdb1b655c0b.png,0
29983,5fb64e85-62ef-4852-8ad1-f799015c8cc3.png,0
29984,c57ad4fd-3aa8-4ae8-a262-7336360d8265.png,0


In [14]:
def compute_mean_std(dataloader):
    mean1 = 0.
    std1 = 0.
    nb_samples = 0.
    for data, _ in tqdm(dataloader):
        # print(data)
        batch_samples = data.size(0)

        # Rearrange data to be the shape of [B, C, W * H]
        data = data.view(batch_samples, data.size(1), -1) # torch.Size([32, 1, 65536])
        
        # Compute mean and std here
        mean1 += data.mean(2).sum(0) # tinh mean theo width*height va lay tong theo batch_size
        # print(data.mean(2).size())
        std1 += data.std(2).sum(0)

        # Update total number of images
        nb_samples += batch_samples
    mean1 /= nb_samples
    std1 /= nb_samples

    return mean1, std1

In [15]:
transfms_M_STD = A.Compose([
    A.Resize(256,256),
    A.Normalize(mean = [0.], std =[1.]), # scale pixel values from [0,255] to [0,1]
    ToTensorV2() 
])

In [17]:
dataset = ImageDataset(train_txt1, '/mnt/DATA/research/classificationCOVID19applyseg/dataset/COVIDxCXR3/train/', transfms_M_STD) # train + val
dataloader = DataLoader(dataset, batch_size=64, shuffle=False)
data, _ = next(iter(dataloader))
mean, std = compute_mean_std(dataloader)
print(mean, std)

100%|██████████| 469/469 [05:12<00:00,  1.50it/s]

tensor([0.5002, 0.5002, 0.5003]) tensor([0.2079, 0.2079, 0.2079])
